In [1]:
import os
import json
import math
from glob import glob
from tqdm import tqdm

import numpy as np
import pandas as pd
import cv2

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split

2025-11-27 19:10:32.792782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764270632.983390     102 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764270633.029135     102 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
DATA_DIR = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
MERGED_MASK_DIR = "/kaggle/working/merged_masks"
WEIGHTS_PATH = "/kaggle/input/efficientnetv2/keras/efficientnetv2_b0_imagenet/3/model.weights.h5"
IMG_SIZE = 384                 
BATCH_SIZE = 8
EPOCHS = 20
LEARNING_RATE = 1e-4
SEED = 42
FORGED_TO_AUTH_RATIO = 0.25    
MIN_INSTANCE_PIXELS = 30      
os.makedirs(MERGED_MASK_DIR, exist_ok=True)

In [3]:
def mask_to_rle_list(mask):
    """
    mask: 2D binary numpy array (H, W) with dtype uint8 or bool
    returns list [start1, len1, start2, len2, ...] (1-indexed, Fortran order)
    """
    # flatten in Fortran order (column-major)
    pixels = mask.T.flatten()  # now column-major flattened
    # find transitions
    padded = np.concatenate([[0], pixels, [0]])
    runs = np.where(padded[1:] != padded[:-1])[0] + 1
    # runs gives starts and ends alternately
    runs[1::2] = runs[1::2] - runs[:-1:2]
    # convert to 1-indexed starts: runs starts are already 1-indexed
    out = runs.tolist()
    return out

In [4]:
def rle_encode_instances(list_of_masks):
    """
    list_of_masks: list of 2D binary masks (H, W)
    returns annotation string: 'authentic' or 'json([...]);json([...]);...'
    """
    if len(list_of_masks) == 0:
        return "authentic"
    parts = []
    for m in list_of_masks:
        if m.dtype != np.uint8 and m.dtype != np.bool_:
            m = (m > 0).astype(np.uint8)
        rle = mask_to_rle_list(m)
        if len(rle) == 0:
            continue
        parts.append(json.dumps(rle))
    if len(parts) == 0:
        return "authentic"
    return ';'.join(parts)

In [5]:
def rle_decode_instance(rle_list, shape):
    """
    rle_list: list [start1, len1, start2, len2...]
    shape: (H, W)
    returns binary mask (H, W)
    """
    if len(rle_list) == 0:
        return np.zeros(shape, dtype=np.uint8)
    H, W = shape
    arr = np.zeros(H*W, dtype=np.uint8)
    r = np.array(rle_list, dtype=np.int64)
    starts = r[0::2] - 1
    lengths = r[1::2]
    for s, l in zip(starts, lengths):
        arr[s:s+l] = 1
    return arr.reshape((W, H)).T  # convert back from Fortran flatten to HxW

In [6]:
def merge_and_save_masks(data_dir, out_dir):
    train_mask_dir = os.path.join(data_dir, 'train_masks')
    supp_mask_dir = os.path.join(data_dir, 'supplemental_masks')
    train_img_dir = os.path.join(data_dir, 'train_images', 'forged')
    supp_img_dir = os.path.join(data_dir, 'supplemental_images')

    # collect mask files
    mask_files = []
    if os.path.isdir(train_mask_dir):
        mask_files += glob(os.path.join(train_mask_dir, '*.npy'))
    if os.path.isdir(supp_mask_dir):
        mask_files += glob(os.path.join(supp_mask_dir, '*.npy'))

    # group by image id (base name before first underscore/dot)
    groups = {}
    for p in mask_files:
        fname = os.path.basename(p)
        img_id = fname.split('.')[0].split('_')[0]
        groups.setdefault(img_id, []).append(p)

    print(f"Found {len(groups)} images with mask files")

    for img_id, paths in tqdm(groups.items()):
        # find original image path to get shape
        candidates = [
            os.path.join(train_img_dir, f"{img_id}.png"),
            os.path.join(supp_img_dir, f"{img_id}.png")
        ]
        img_path = None
        for c in candidates:
            if os.path.exists(c):
                img_path = c
                break
        if img_path is None:
            # some masks might belong to authentic images or missing images, skip
            continue

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        H, W = img.shape
        inst_list = []

        for p in paths:
            arr = np.load(p, allow_pickle=False)
            # arr may be shape (N,H,W) or (H,W) or (C,H,W)
            if arr.ndim == 3:
                # common: (N,H,W) or (C,H,W)
                # decide: if first dim is small (<6) treat as instances stack
                if arr.shape[0] <= 10:  # heuristic
                    instances = [arr[i] for i in range(arr.shape[0])]
                else:
                    # sometimes (H,W,C) case: collapse channels
                    instances = [arr.max(axis=0)]
            elif arr.ndim == 2:
                instances = [arr]
            elif arr.ndim == 4:
                # improbable, but collapse first axis
                instances = [arr[i].max(axis=0) for i in range(arr.shape[0])]
            else:
                instances = [arr.reshape(arr.shape[-2], arr.shape[-1])]

            for inst in instances:
                if inst.ndim == 3:
                    inst = inst.max(axis=0)
                # binarize
                inst_bin = (inst > 0).astype(np.uint8)
                # resize instance to original image shape (safe nearest)
                if inst_bin.shape != (H, W):
                    inst_bin = cv2.resize(inst_bin.astype(np.uint8), (W, H), interpolation=cv2.INTER_NEAREST)
                    inst_bin = (inst_bin > 0).astype(np.uint8)
                if inst_bin.sum() < MIN_INSTANCE_PIXELS:
                    continue
                inst_list.append(inst_bin)

        if len(inst_list) == 0:
            # nothing left
            continue

        # Save stacked masks as uint8 array (N, H, W)
        stacked = np.stack(inst_list, axis=0).astype(np.uint8)
        outp = os.path.join(out_dir, f"{img_id}.npy")
        np.save(outp, stacked)

    print("Merged masks saved to:", out_dir)

In [7]:
merge_and_save_masks(DATA_DIR, MERGED_MASK_DIR)

Found 2795 images with mask files


100%|██████████| 2795/2795 [02:29<00:00, 18.75it/s]

Merged masks saved to: /kaggle/working/merged_masks


In [8]:
train_forged_dir = os.path.join(DATA_DIR, 'train_images', 'forged')
train_auth_dir = os.path.join(DATA_DIR, 'train_images', 'authentic')

In [9]:
forged_files = sorted([f for f in os.listdir(train_forged_dir) if f.endswith('.png')])
auth_files = sorted([f for f in os.listdir(train_auth_dir) if f.endswith('.png')])

In [10]:
# forged that actually have masks
forged_with_mask = []
forged_mask_paths = []
for f in forged_files:
    img_id = f.split('.')[0]
    maskp = os.path.join(MERGED_MASK_DIR, f"{img_id}.npy")
    if os.path.exists(maskp):
        forged_with_mask.append(os.path.join(train_forged_dir, f))
        forged_mask_paths.append(maskp)

In [11]:
auth_paths = [os.path.join(train_auth_dir, f) for f in auth_files]

In [12]:
print("Forged with masks:", len(forged_with_mask))
print("Authentic images:", len(auth_paths))

Forged with masks: 2751
Authentic images: 2377


In [13]:
# Compose dataset with desired ratio: for training we'll generate balanced batches via sampling later.
samples = []
# add forged
for imgp, m in zip(forged_with_mask, forged_mask_paths):
    samples.append((imgp, m, 1))
# add authentic
for p in auth_paths:
    samples.append((p, None, 0))

print("Total samples (raw):", len(samples))

Total samples (raw): 5128


In [14]:
# Shuffle samples
rng = np.random.default_rng(SEED)
rng.shuffle(samples)

In [15]:
# Train/val split
train_samples, val_samples = train_test_split(samples, test_size=0.1, random_state=SEED, stratify=[s[2] for s in samples])
print("Train:", len(train_samples), "Val:", len(val_samples))

Train: 4615 Val: 513


In [16]:
AUTOTUNE = tf.data.AUTOTUNE

In [17]:
def read_image_gray_resize(path, img_size=IMG_SIZE):
    img = cv2.imread(path.decode('utf-8'), cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(path)
    img_resized = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_LINEAR)
    img_resized = img_resized.astype(np.float32) / 255.0
    return img_resized.astype(np.float32)

In [18]:
def read_mask_stack_resize(mask_path, img_size=IMG_SIZE):
    # mask_path may be 'None' for authentic images
    if mask_path.decode('utf-8') == 'None':
        return np.zeros((img_size, img_size), dtype=np.float32)
    arr = np.load(mask_path.decode('utf-8'), allow_pickle=False)  # (N, H, W)
    if arr.ndim == 3:
        # create single-channel instance-aggregated target (for training use can be multi-instance but model predicts one binary mask)
        # For training we use a single merged mask (semantic), but we keep instance masks in merging step for submission.
        merged = np.maximum.reduce(arr)
    else:
        merged = arr
    merged = (merged > 0).astype(np.float32)
    merged_resized = cv2.resize(merged, (img_size, img_size), interpolation=cv2.INTER_NEAREST)
    return merged_resized.astype(np.float32)

In [19]:
def _load_pair_py(image_path, mask_path):
    # used inside tf.numpy_function, so inputs are bytes
    img = read_image_gray_resize(image_path, IMG_SIZE)
    mask = read_mask_stack_resize(mask_path, IMG_SIZE)
    # add channel dim
    img = img[..., None].astype(np.float32)
    mask = mask[..., None].astype(np.float32)
    return img, mask

In [20]:
def dataset_from_samples(sample_list, batch_size=BATCH_SIZE, shuffle=True, augment=True):
    """
    We will implement a sampling strategy to approximate FORGED_TO_AUTH_RATIO in batches:
    - build two datasets: forged_ds and auth_ds
    - sample from them to compose the final dataset.
    """
    forged = [s for s in sample_list if s[2] == 1]
    auth = [s for s in sample_list if s[2] == 0]

    # Build tf.data for forged and auth
    def build_simple_ds(samples):
        image_paths = [s[0] for s in samples]
        mask_paths = [s[1] if s[1] is not None else 'None' for s in samples]
        ds = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
        if shuffle:
            ds = ds.shuffle(len(image_paths), seed=SEED)
        # map to tensors
        def _tf_map(image_path, mask_path):
            img, mask = tf.numpy_function(_load_pair_py, [image_path, mask_path], [tf.float32, tf.float32])
            img.set_shape([IMG_SIZE, IMG_SIZE, 1])
            mask.set_shape([IMG_SIZE, IMG_SIZE, 1])
            return img, mask
        ds = ds.map(_tf_map, num_parallel_calls=AUTOTUNE)
        return ds

    forged_ds = build_simple_ds(forged)
    auth_ds = build_simple_ds(auth)

    # Determine per-batch counts
    forged_per_batch = max(1, int(BATCH_SIZE * FORGED_TO_AUTH_RATIO))
    auth_per_batch = BATCH_SIZE - forged_per_batch

    # Create windows/batches by zipping repeated datasets
    forged_ds = forged_ds.repeat()
    auth_ds = auth_ds.repeat()

    # batch each temporarily into forged_per_batch and auth_per_batch
    forged_ds = forged_ds.batch(forged_per_batch)
    auth_ds = auth_ds.batch(auth_per_batch)

    # zip and flatten
    ds = tf.data.Dataset.zip((forged_ds, auth_ds))

    def _merge_batches(f_batch, a_batch):
        # f_batch: (f_b, IMG, IMG,1), (f_b, IMG, IMG,1)
        f_imgs, f_masks = f_batch
        a_imgs, a_masks = a_batch
        imgs = tf.concat([f_imgs, a_imgs], axis=0)
        masks = tf.concat([f_masks, a_masks], axis=0)

        # optionally apply tf augmentations here
        # random flip
        imgs_masks = tf.concat([imgs, masks], axis=-1)  # shape (B, H, W, 2)
        # random horizontal and vertical flip
        imgs_masks = tf.image.random_flip_left_right(imgs_masks)
        imgs_masks = tf.image.random_flip_up_down(imgs_masks)
        # random brightness/contrast on images channel only
        imgs = imgs_masks[..., 0:1]
        masks = imgs_masks[..., 1:2]
        imgs = tf.image.random_brightness(imgs, max_delta=0.1)
        imgs = tf.image.random_contrast(imgs, 0.9, 1.1)
        # clip
        imgs = tf.clip_by_value(imgs, 0.0, 1.0)
        # shuffle within batch
        idx = tf.random.shuffle(tf.range(tf.shape(imgs)[0]))
        imgs = tf.gather(imgs, idx)
        masks = tf.gather(masks, idx)
        return imgs, masks

    ds = ds.map(_merge_batches, num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(AUTOTUNE)
    return ds

In [21]:
train_ds = dataset_from_samples(train_samples, batch_size=BATCH_SIZE, shuffle=True, augment=True)
val_ds = dataset_from_samples(val_samples, batch_size=BATCH_SIZE, shuffle=False, augment=False)

I0000 00:00:1764270796.631584     102 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [22]:
def to_rgb(image):
    if image.shape[-1] == 1:
        image = tf.image.grayscale_to_rgb(image)
    return image

In [23]:
train_ds = train_ds.map(lambda x, y: (to_rgb(x), y),
                        num_parallel_calls=tf.data.AUTOTUNE)

val_ds = val_ds.map(lambda x, y: (to_rgb(x), y),
                    num_parallel_calls=tf.data.AUTOTUNE)


In [24]:
def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    return x

In [25]:
def build_efficient_unet(IMG_SIZE=512, dropout_rate=0.2):
    """
    EfficientNetV2B0-based U-Net.
    Uses stable layer names instead of fragile index-based skips.
    """

    # Load without pretrained weights to avoid network call
    base = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
        include_top=False,
        include_preprocessing=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        weights=None  # Changed from 'imagenet' to None
    )

    # Stable skip connections — these layer names exist across TF versions
    skip1 = base.get_layer("block2b_add").output   # ~128x128
    skip2 = base.get_layer("block3b_add").output   # ~64x64
    skip3 = base.get_layer("block5c_add").output   # ~32x32
    encoder_output = base.get_layer("top_activation").output  # ~16x16

    # Start decoder
    x = encoder_output

    # Block 1: upsample to match skip3
    x = layers.Conv2DTranspose(256, 3, strides=2, padding="same")(x)
    x = layers.Concatenate()([x, skip3])
    x = layers.Conv2D(256, 3, padding="same", activation="relu")(x)
    x = layers.Conv2D(256, 3, padding="same", activation="relu")(x)

    # Block 2: upsample to match skip2
    x = layers.Conv2DTranspose(128, 3, strides=2, padding="same")(x)
    x = layers.Concatenate()([x, skip2])
    x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)

    # Block 3: upsample to match skip1
    x = layers.Conv2DTranspose(64, 3, strides=2, padding="same")(x)
    x = layers.Concatenate()([x, skip1])
    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)

    # Block 4: upsample towards final output
    x = layers.Conv2DTranspose(32, 3, strides=2, padding="same")(x)
    x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
    x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)

    # Block 5: final upsample (to full resolution)
    x = layers.Conv2DTranspose(16, 3, strides=2, padding="same")(x)
    x = layers.Conv2D(16, 3, padding="same", activation="relu")(x)

    # Output mask
    outputs = layers.Conv2D(1, 1, activation="sigmoid")(x)

    # Build model
    model = models.Model(inputs=base.input, outputs=outputs)

    return model

In [26]:
model = build_efficient_unet(IMG_SIZE=IMG_SIZE)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 384, 384,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 192, 192,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 192, 192,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 192, 192,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 192, 192,  │      4,608 │ stem_activation[… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 192, 192,  │         64 │ block1a_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 192, 192,  │          0 │ block1a_project_… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 96, 96,    │      9,216 │ block1a_project_… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 96, 96,    │        256 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 96, 96,    │          0 │ block2a_expand_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_co… │ (None, 96, 96,    │      2,048 │ block2a_expand_a… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_bn  │ (None, 96, 96,    │        128 │ block2a_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_conv │ (None, 96, 96,    │     36,864 │ block2a_project_… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_bn   │ (None, 96, 96,    │        512 │ block2b_expand_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_act… │ (None, 96, 96,    │          0 │ block2b_expand_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_project_co… │ (None, 96, 96,    │      4,096 │ block2b_expand_a… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_project_bn  │ (None, 96, 96,    │        128 │ block2b_project_

 Total params: 11,162,401 (42.58 MB)

 Trainable params: 11,101,793 (42.35 MB)

 Non-trainable params: 60,608 (236.75 KB)

In [27]:
smooth = 1e-6

In [28]:
def dice_coef(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    inter = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * inter + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

In [29]:
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [30]:
def bce_dice(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return 0.5 * bce + 0.5 * dice_loss(y_true, y_pred)

In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=bce_dice,
              metrics=[dice_coef])

In [32]:
checkpoint_path = '/kaggle/working/best_model.keras'
cb = [
    callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss', mode='min'),
    callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
]


In [33]:
steps_per_epoch = len(train_samples) // BATCH_SIZE
validation_steps = len(val_samples) // BATCH_SIZE

history = model.fit(
    train_ds,
    validation_data=val_ds,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=EPOCHS,
    callbacks=cb,
    verbose=1
)


Epoch 1/20


I0000 00:00:1764270842.977979     154 service.cc:148] XLA service 0x7c7c341c68a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764270842.978467     154 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1764270849.570277     154 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1764270886.697191     154 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


576/576 ━━━━━━━━━━━━━━━━━━━━ 211s 218ms/step - dice_coef: 0.0234 - loss: 0.5649 - val_dice_coef: 0.0370 - val_loss: 0.5159 - learning_rate: 1.0000e-04
Epoch 2/20
576/576 ━━━━━━━━━━━━━━━━━━━━ 115s 199ms/step - dice_coef: 0.0466 - loss: 0.5183 - val_dice_coef: 0.0620 - val_loss: 0.5054 - learning_rate: 1.0000e-04
Epoch 3/20
383/576 ━━━━━━━━━━━━━━━━━━━━ 34s 179ms/step - dice_coef: 0.0646 - loss: 0.5085

KeyboardInterrupt: 

In [34]:
model.save("efficient_unet.keras")

In [ ]:
# model.save_weights("efficient_unet.weights.h5")

In [35]:
test_dir = os.path.join(DATA_DIR, 'test_images')
test_files = sorted([f for f in os.listdir(test_dir) if f.endswith('.png')])

In [36]:
submission_rows = []

In [37]:
for fname in tqdm(test_files):
    img_id = fname.split('.')[0]
    img_path = os.path.join(test_dir, fname)
    orig = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if orig is None:
        # skip
        submission_rows.append([img_id, "authentic"])
        continue
    H, W = orig.shape

    # preprocess for model - CONSISTENT WITH TRAINING
    img_resized = cv2.resize(orig, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)
    img_resized = img_resized.astype(np.float32) / 255.0
    
    # Add channel dimension and convert to RGB (same as training)
    img_resized = img_resized[..., None]  # Add channel dimension: (IMG_SIZE, IMG_SIZE, 1)
    img_resized_rgb = np.repeat(img_resized, 3, axis=-1)  # Convert to RGB: (IMG_SIZE, IMG_SIZE, 3)
    
    # Create batch dimension
    inp = img_resized_rgb[None, ...]  # Shape: (1, IMG_SIZE, IMG_SIZE, 3)

    pred = model.predict(inp, verbose=0)[0, ..., 0]
    # threshold
    pred_bin = (pred > 0.5).astype(np.uint8)

    # connected components at model resolution (keeps instance separation)
    num_labels, labels = cv2.connectedComponents(pred_bin, connectivity=8)
    instances = []
    for lab in range(1, num_labels):
        inst = (labels == lab).astype(np.uint8)
        if inst.sum() < 8:  # tiny spurious at model size -> skip
            continue
        # resize back to original image resolution per-instance
        inst_back = cv2.resize(inst, (W, H), interpolation=cv2.INTER_NEAREST)
        inst_back = (inst_back > 0).astype(np.uint8)
        if inst_back.sum() < MIN_INSTANCE_PIXELS:
            continue
        instances.append(inst_back)

    # if no instance found -> authentic
    if len(instances) == 0:
        submission_rows.append([img_id, "authentic"])
    else:
        # ensure instances are non-overlapping (optional): subtract overlaps by priority (large first)
        instances_sorted = sorted(instances, key=lambda x: x.sum(), reverse=True)
        final_instances = []
        occupied = np.zeros((H, W), dtype=np.uint8)
        for inst in instances_sorted:
            inst2 = inst.copy()
            inst2[occupied == 1] = 0
            if inst2.sum() < MIN_INSTANCE_PIXELS:
                continue
            final_instances.append(inst2)
            occupied = np.maximum(occupied, inst2)

        annotation = rle_encode_instances(final_instances)
        submission_rows.append([img_id, annotation])

100%|██████████| 1/1 [00:07<00:00,  7.06s/it]


In [38]:
sub_df = pd.DataFrame(submission_rows, columns=['case_id', 'annotation'])
sub_df.to_csv('submission.csv', index=False)
print("Saved submission.csv with rows:", len(sub_df))

Saved submission.csv with rows: 1


In [39]:
for i, r in sub_df.iterrows():
    if r['annotation'] != 'authentic':
        ann = r['annotation']
        parts = ann.split(';')
        print("Example decoded masks for", r['case_id'], "->", len(parts), "instances")
        break

In [40]:
sub_df.head()

,case_id,annotation
0,45,authentic


In [41]:
print("Notebook run complete.")

Notebook run complete.
